In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random 

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score
import keras_tuner as kt

In [31]:
df = pd.read_csv("datasets\covid19postvaxstatewidestats.csv")

df = df[df['unvaccinated_cases_per_100k'].notna()]

df = df.reset_index(drop=True)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\dangn\AppData\Local\Temp\ipykernel_12440\2108600221.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv("datasets\covid19postvaxstatewidestats.csv")


In [32]:
df_unvac = df[['unvaccinated_cases_per_100k', 'unvaccinated_hosp_per_100k', 'unvaccinated_deaths_per_100k']]

df_unvac

,unvaccinated_cases_per_100k,unvaccinated_hosp_per_100k,unvaccinated_deaths_per_100k
0,27.251295,1.483401,0.067702
1,25.072362,1.360519,0.078033
2,23.268146,1.296892,0.087677
3,21.766329,1.236441,0.099877
4,20.467271,1.179528,0.112219
...,...,...,...
778,8.425650,0.613146,0.051096
779,8.210084,0.564715,0.048550
780,8.268116,0.552060,0.048561
781,8.157356,0.570069,0.061353


In [33]:
def sequence_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :])
        y.append(data[i+time_step,:])
    return np.array(X), np.array(y)

In [34]:
train_data = df_unvac.values

In [35]:
X_train, y_train = sequence_data(train_data, 10)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])

In [36]:
X_train.shape

(772, 10, 3)

In [37]:
y_train.shape

(772, 3)

In [38]:
def model_builder(hp):
    model = Sequential()
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)

    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(LSTM(units = hp_layer_1, activation=hp_activation, input_shape = (10,3)))

    model.add(Dense(units = hp_layer_2, activation = hp_activation))
    
    model.add(Dense(units=3, activation=hp_activation))


    model.compile(optimizer = Adam(learning_rate=hp_learning_rate), loss='mse', metrics=['mae'])

    return model

In [39]:
tuner = kt.Hyperband(model_builder, 
                     objective='val_mae',
                     max_epochs=10,
                     factor = 3,
                     directory = 'dir',
                     project_name = 'tune')

Reloading Tuner from dir\tune\tuner0.json


In [40]:
stop_early = EarlyStopping(monitor = 'val_mae', patience = 3)

In [41]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

In [42]:
#One hidden layer - 0.7743412256240845
#Two hidden layer - 